<a href="https://colab.research.google.com/github/luaiabuelsamen/McHacks9-TMIL/blob/master/NLP_Twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cryptocurreny Market Optimism analyser

In [39]:
!pip install transformers==4.15.0
!pip install searchtweets

In [40]:
!pip install tweepy==4.4.0

1. Import libraries and dependencies

In [48]:
import transformers
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch import softmax
import numpy as np
import pandas as pd

2. Load in pre-trained sentiment (positivity) prediction model


In [127]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

3. Tweets import and pre-processing

In [72]:
import tweepy
consumer_key = "2xahjEG9TT3v3lV2kFCdFAEjt"
consumer_secret = "LuYQEXzpBRl0J3EIAYeB9t61rt7wyhMnvgFHJtWKbHUG8uib3m"
bearer_token = "AAAAAAAAAAAAAAAAAAAAAJSlYQEAAAAAr0ClDhRusUwMfk5JC5N4n6HPRvE%3DsEVzWdUqW3mGF6uOCgt1umA3Dc7XOZEwA104RNBzSQSyybGAS8"
auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)

client = tweepy.Client(bearer_token, consumer_key, consumer_secret)
t = tweepy.Paginator(client.search_recent_tweets, query='ETH',max_results=10).flatten(limit=10)

In [73]:
tweets = []
for tweet in t:
    tweets.append(tweet.text)

print(tweets)

['@sn1pefx 1/1 0.02 eth https://t.co/ZSzV43n0EZ', 'RT @cz_binance: .@VitalikButerin just thanked me for the 2nd time in the last 3 months. Let’s build together. #eth #bnb 🤝', 'RT @0xAlyssaBelle: I ‘ll be giving $1000  $ETH  to the first 2999 people who follows, retweets and joins https://t.co/oQhK2sYxV5!!💸💸💸', '@Union_gal12 🐋𝐇𝐎𝐑𝐆𝐈 𝐓𝐎𝐊𝐄𝐍🐋\n\n🖼1ST COLLECTION NFTS RELEASED. 1000 PIECES, 5 RARITIES\nMINT FEE 0.12 BNB \n https://t.co/COU4mNpJNu\n\n✅ETH/FTM\n✅CEX LISTINGS ON THE WAY\nTokenomic\n✅10% tax \n✅5% marketing\n✅2% staking \nWebsite: https://t.co/cqh6Rr6e0A\nTelegram: https://t.co/IdBqz0ob1c', 'RT @ETHGifting: 7000$ $ETH —  7 Winners 🏆\n                               \nTo Enter:                \n1️⃣ Like, RT &amp; Follow \n2️⃣ Drop Wallet…', 'RT @LauraVGems: 🚀Hold exchange or break your #NFT to obtain rewards with this revolutionary #NFTProject\n\n🐷 @metapiggybanks\n🔥Burn system\n🔥ET…', 'RT @RetsuNFT: Will send this @HAPEBEASTGANG to one random person, 6.6 $ETH ($20,628 at the time 

In [164]:
import re
def preprocess(tweets):
    tweet_lst = pd.DataFrame(tweets)
    tweet_lst['text'] = tweet_lst[0]
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # flags
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", flags=re.UNICODE)
    lines = lambda x: re.sub("\n", " ",x)
    emoji = lambda x: re.sub(emoji_pattern, " ",x)
    remove_rt = lambda x: re.sub('RT @\w+: '," ",x)
    rt = lambda x: re.sub("(@[A-Za-z0–9_]+)|([\t])|(\w+:\/\/\S+)"," ",x)
    tweet_lst['text'] = tweet_lst.text.map(remove_rt).map(rt).map(emoji).map(lines)
    return tweet_lst


4. Score tweets based on sentiment prediction

In [171]:
def pred(tweet):
    tokens = tokenizer.encode(tweet, return_tensors='pt')
    result = model(tokens)
    return np.arange(1,6) @ result.logits.softmax(dim=1).tolist()[0]

def get_score(lst):
    lst['score'] = np.NaN
    for index, row in lst.iterrows():
        # print(lst.text[index])
        lst['score'][index] = pred(lst['text'][index])
    return lst

get_score(preprocess(tweets))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,0,text,score
0,@sn1pefx 1/1 0.02 eth https://t.co/ZSzV43n0EZ,1pefx 1/1 0.02 eth,2.031967
1,RT @cz_binance: .@VitalikButerin just thanked ...,. just thanked me for the 2nd time in the la...,4.404743
2,RT @0xAlyssaBelle: I ‘ll be giving $1000 $ETH...,I ‘ll be giving $1000 $ETH to the first 299...,1.631643
3,@Union_gal12 🐋𝐇𝐎𝐑𝐆𝐈 𝐓𝐎𝐊𝐄𝐍🐋\n\n🖼1ST COLLECTION ...,12 1ST COLLECTION NFTS RELEASED. 1000 P...,2.117309
4,RT @ETHGifting: 7000$ $ETH — 7 Winners 🏆\n ...,7000$ $ETH — 7 Winners ...,2.583581
5,RT @LauraVGems: 🚀Hold exchange or break your #...,Hold exchange or break your #NFT to obtain r...,2.672315
6,RT @RetsuNFT: Will send this @HAPEBEASTGANG to...,"Will send this to one random person, 6.6 $E...",1.495775
7,RT @MrsunNFT: Drop your ETH address. Randomly ...,Drop your ETH address. Randomly drop NFT ...,1.297078
8,RT @cryprules: $200 | 2.8 JT | iPhone 6s • 4 D...,"$200 | 2.8 JT | iPhone 6s • 4 Days retweet,...",2.205484
9,@sailormarsnft Check it out The Special CALM C...,Check it out The Special CALM CAT! CALM CAT...,2.075135


In [173]:
print(get_score(preprocess(tweets))['score'].mean())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


2.251503081433475
